In [47]:
import pandas as pd
from configurations import basic_parameters as bp

In [48]:
csv_dataframe = bp['dataframe_dir'] + '/' + bp['dataframe_file']

In [49]:
names = ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']

df = pd.read_csv(csv_dataframe, names=names, usecols=['Open time', 'Open', 'High', 'Low', 'Close'])

In [50]:
def computeRSI (data, time_window):
    diff = data.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]
    
    # check pandas documentation for ewm
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html
    # values are related to exponential decay
    # we set com=time_window-1 so we get decay alpha=1/time_window
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

In [51]:
def stochastic(data, k_window, d_window, window):
    
    # input to function is one column from df
    # containing closing price or whatever value we want to extract K and D from
    
    min_val = data.rolling(window=window, center=False).min()
    max_val = data.rolling(window=window, center=False).max()
    
    stoch = ( (data - min_val) / (max_val - min_val) ) * 100
    
    K = stoch.rolling(window=k_window, center=False).mean() 
    #K = stoch
    
    D = K.rolling(window=d_window, center=False).mean() 


    return K, D

In [52]:
df['RSI'] = computeRSI(df['Close'], 14)
df['K'], df['D'] = stochastic(df['RSI'], 3, 3, 14)

In [53]:
df['time UTC'] = pd.to_datetime(df['Open time'], unit='ms', origin='unix')
df.dropna(inplace=True)
df

,Open time,Open,High,Low,Close,RSI,K,D,time UTC
31,1678060831000,22422.12,22423.83,22421.52,22422.54,54.033107,91.624977,95.381293,2023-03-06 00:00:31
32,1678060832000,22422.54,22424.50,22422.31,22422.54,54.033107,88.731052,91.624977,2023-03-06 00:00:32
33,1678060833000,22422.55,22422.73,22422.52,22422.55,54.080566,91.371062,90.575697,2023-03-06 00:00:33
34,1678060834000,22422.53,22423.00,22422.32,22422.32,52.732041,89.725047,89.942387,2023-03-06 00:00:34
35,1678060835000,22422.46,22423.01,22422.16,22422.16,51.765031,86.953478,89.349862,2023-03-06 00:00:35
...,...,...,...,...,...,...,...,...,...
86395,1678147195000,22408.90,22409.75,22408.41,22408.53,50.694848,6.975464,12.929896,2023-03-06 23:59:55
86396,1678147196000,22408.96,22409.84,22408.54,22409.72,58.654142,16.822415,10.257781,2023-03-06 23:59:56
86397,1678147197000,22409.28,22409.28,22408.89,22408.89,52.311174,16.042611,13.280163,2023-03-06 23:59:57
86398,1678147198000,22409.61,22409.73,22408.90,22408.90,52.377994,18.862486,17.242504,2023-03-06 23:59:58


In [54]:
#df.loc[(df['D'] <= 0) & (df['RSI'] < 30), 'buy?'] = True
df.loc[df['D'] <= 5, 'buy?'] = True
df.loc[df['buy?'] == True]

,Open time,Open,High,Low,Close,RSI,K,D,time UTC,buy?
96,1678060896000,22430.54,22431.47,22430.54,22430.98,44.741408,2.993217,3.014704,2023-03-06 00:01:36,True
149,1678060949000,22431.48,22431.48,22430.23,22431.41,36.119542,5.418841,3.352601,2023-03-06 00:02:29,True
150,1678060950000,22430.12,22430.90,22428.92,22429.56,29.520583,4.631871,4.483887,2023-03-06 00:02:30,True
151,1678060951000,22429.86,22429.94,22428.98,22429.56,29.520583,2.017890,4.022867,2023-03-06 00:02:31,True
152,1678060952000,22429.81,22429.90,22428.74,22429.85,31.786274,2.745612,3.131791,2023-03-06 00:02:32,True
...,...,...,...,...,...,...,...,...,...,...
85985,1678146785000,22409.64,22410.00,22408.98,22409.88,35.561208,6.790041,4.859099,2023-03-06 23:53:05,True
86210,1678147010000,22410.84,22411.00,22410.73,22410.90,34.995551,0.000000,2.849722,2023-03-06 23:56:50,True
86211,1678147011000,22410.99,22411.33,22410.92,22411.32,39.288821,4.718490,1.572830,2023-03-06 23:56:51,True
86329,1678147129000,22409.29,22409.92,22408.84,22408.87,39.779264,3.530634,3.928294,2023-03-06 23:58:49,True


In [55]:
rolling_seconds = 60

df['min_val'] = df['Low'].shift(-rolling_seconds).rolling(window=rolling_seconds).min()
df['max_val'] = df['High'].shift(-rolling_seconds).rolling(window=rolling_seconds).max()

df['max_val_%'] = (df['max_val'] - df['Open'].shift(-1)) * 100 / df['Open'].shift(-1)
df['min_val_%'] = (df['min_val'] - df['Open'].shift(-1)) * 100 / df['Open'].shift(-1)

#df.dropna(inplace=True)
#df[60:70]
#df.loc[df['buy?'] == True]
df = df.loc[df['buy?'] == True]
df = df.loc[df['Open time'] - df['Open time'].shift(1) > rolling_seconds * 1000]
df

,Open time,Open,High,Low,Close,RSI,K,D,time UTC,buy?,min_val,max_val,max_val_%,min_val_%
283,1678061083000,22430.58,22430.84,22430.54,22430.84,36.869953,4.649163,2.584153,2023-03-06 00:04:43,True,22418.59,22435.84,0.022692,-0.054211
641,1678061441000,22422.72,22422.88,22422.42,22422.56,29.965102,6.799701,3.794018,2023-03-06 00:10:41,True,22413.56,22424.42,0.008251,-0.040183
752,1678061552000,22417.72,22418.07,22417.50,22417.83,38.985795,10.217537,3.405846,2023-03-06 00:12:32,True,22413.41,22421.30,0.016996,-0.018200
982,1678061782000,22405.37,22405.48,22402.76,22402.76,27.958715,2.413241,2.889133,2023-03-06 00:16:22,True,22361.43,22403.39,0.002901,-0.184397
1104,1678061904000,22387.44,22387.44,22383.50,22384.82,46.983858,0.134929,4.061707,2023-03-06 00:18:24,True,22382.87,22394.54,0.043288,-0.008845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85498,1678146298000,22407.34,22408.05,22406.76,22408.01,45.592859,8.462599,3.374215,2023-03-06 23:44:58,True,22398.22,22409.93,0.008524,-0.043734
85654,1678146454000,22407.42,22407.50,22403.16,22403.96,34.752357,3.109736,3.430145,2023-03-06 23:47:34,True,22402.00,22409.41,0.019862,-0.013211
85949,1678146749000,22413.06,22413.84,22412.46,22413.50,56.561471,5.492844,4.315523,2023-03-06 23:52:29,True,22408.98,22418.68,0.021817,-0.021460
86210,1678147010000,22410.84,22411.00,22410.73,22410.90,34.995551,0.000000,2.849722,2023-03-06 23:56:50,True,22408.65,22412.72,0.007719,-0.010441


In [56]:
tp_size = 0.075
sl_size = 0.075
# 0.075 RSI 40 total profit 2.47

In [57]:
sl = df.loc[(df['buy?'] == True) & (df['min_val_%'] < -sl_size)]
#sl = sl.loc[sl['Open time'] - sl['Open time'].shift(1) > rolling_seconds * 1000]
sl

,Open time,Open,High,Low,Close,RSI,K,D,time UTC,buy?,min_val,max_val,max_val_%,min_val_%
982,1678061782000,22405.37,22405.48,22402.76,22402.76,27.958715,2.413241,2.889133,2023-03-06 00:16:22,True,22361.43,22403.39,0.002901,-0.184397
5081,1678065881000,22497.41,22499.97,22488.89,22493.44,71.357009,3.320695,3.518161,2023-03-06 01:24:41,True,22462.01,22508.00,0.066065,-0.138398
6798,1678067598000,22394.00,22394.42,22391.12,22392.35,42.855804,0.883917,4.717182,2023-03-06 01:53:18,True,22322.34,22392.83,0.002233,-0.312562
7296,1678068096000,22365.26,22365.80,22363.69,22363.69,20.083231,0.000000,4.065652,2023-03-06 02:01:36,True,22344.01,22365.33,0.007154,-0.088178
7708,1678068508000,22359.91,22362.58,22356.58,22360.24,52.020573,0.000000,0.833108,2023-03-06 02:08:28,True,22339.25,22371.75,0.049328,-0.096017
8243,1678069043000,22338.92,22339.13,22335.84,22337.13,39.287211,0.000000,0.700117,2023-03-06 02:17:23,True,22315.25,22344.40,0.034114,-0.096388
8819,1678069619000,22343.28,22345.09,22341.36,22343.98,23.302169,7.143746,4.516284,2023-03-06 02:26:59,True,22327.03,22350.90,0.026404,-0.080421
48766,1678109566000,22431.22,22431.97,22430.53,22431.86,63.026207,3.299196,4.344536,2023-03-06 13:32:46,True,22400.30,22432.63,0.003522,-0.140604
49279,1678110079000,22445.31,22445.94,22444.15,22444.97,58.175291,2.944290,3.765670,2023-03-06 13:41:19,True,22421.77,22445.43,0.002005,-0.103408
49602,1678110402000,22479.40,22479.40,22477.09,22477.09,47.795757,1.254151,4.981932,2023-03-06 13:46:42,True,22450.87,22477.85,0.003114,-0.116919


In [58]:
tp = df.loc[(df['buy?'] == True) & (df['max_val_%'] > tp_size)]
#tp = tp.loc[tp['Open time'] - tp['Open time'].shift(1) >  rolling_seconds * 1000]
tp[:50]

,Open time,Open,High,Low,Close,RSI,K,D,time UTC,buy?,min_val,max_val,max_val_%,min_val_%
4614,1678065414000,22389.22,22390.00,22388.19,22389.15,35.894219,4.599040,4.189808,2023-03-06 01:16:54,True,22381.51,22409.48,0.093485,-0.031445
4989,1678065789000,22393.15,22394.18,22393.04,22393.04,51.824733,0.000000,1.594722,2023-03-06 01:23:09,True,22384.03,22419.20,0.114005,-0.043048
5207,1678066007000,22455.04,22456.82,22452.93,22453.85,32.303770,0.074047,1.336308,2023-03-06 01:26:47,True,22445.23,22472.19,0.085735,-0.034338
5968,1678066768000,22410.55,22411.02,22410.32,22410.90,42.737679,1.678726,2.582271,2023-03-06 01:39:28,True,22409.74,22434.42,0.105038,-0.005087
6930,1678067730000,22346.53,22347.42,22344.04,22345.90,38.623676,7.068218,4.576472,2023-03-06 01:55:30,True,22336.38,22364.94,0.085206,-0.042603
7949,1678068749000,22326.30,22326.61,22322.76,22324.14,38.145267,0.000000,3.730974,2023-03-06 02:12:29,True,22320.00,22348.76,0.115576,-0.013260
8475,1678069275000,22323.74,22324.52,22321.76,22322.54,34.419167,0.000000,4.231691,2023-03-06 02:21:15,True,22321.88,22345.07,0.098822,-0.005062
8587,1678069387000,22346.64,22347.73,22345.55,22346.11,41.470228,0.000000,1.584094,2023-03-06 02:23:07,True,22346.10,22373.30,0.119436,-0.002282
9425,1678070225000,22338.62,22340.45,22338.62,22340.45,43.108549,6.562389,4.297392,2023-03-06 02:37:05,True,22335.79,22388.00,0.216207,-0.017502
9593,1678070393000,22375.23,22377.11,22375.22,22377.04,43.260438,4.617762,4.983338,2023-03-06 02:39:53,True,22373.87,22402.74,0.114805,-0.014211


In [59]:
tp.shape[0]/sl.shape[0]

1.2222222222222223

In [60]:
(tp.shape[0] * tp_size) - (sl.shape[0] * sl_size)

0.30000000000000004